# Modelo del recomedación

## Importación de bases

### Importación de librerías

In [1]:
! pip install cython
! pip install wheel
! pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162726 sha256=ba63323fb47a8b8b97f7e1a4e0a73bb9d41d91ba80598c2a919f6df4533acf9e
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [50]:
# Importación de librerías
import pandas as pd
import numpy as np

from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise import dump

### Importación de bases de datos

In [4]:
# Este notebook fue trabajado desde Google Collabs, debido a dificultades para installar la librería surprise en el computador, por lo que se presentan rutas de archivo diferente para tratar de cargar las bases.

# Desde la pc
# base = pd.read_parquet("../Data/final/total_generos.parquet")

# Reviews
# reviews = pd.read_parquet("../Data/final/reviews_final.parquet")
# Games
# games = pd.read_parquet("../Data/final/games_final.parquet")
# items
# items = pd.read_parquet("../Data/final/items_final.parquet")

# En Google Colabs
# Reviews
reviews = pd.read_parquet("/content/reviews_final.parquet")
# Games
games = pd.read_parquet("/content/games_final.parquet")
# items
items = pd.read_parquet("/content/items_final.parquet")


## Creación del modelo de recomendación

In [45]:
# Se revisa la base de datos de review para que el dataframe cumpla con las
# estructuras que se pasan a surprise para crear un modelo de recomendación.
r = reviews[["user_id","item_id","sentiment_analysis"]]
r

,user_id,item_id,sentiment_analysis
0,76561197970982479,1250,2
1,76561197970982479,22200,1
4,js41637,227300,2
5,js41637,239030,2
6,evcentric,248820,1
...,...,...,...
57629,76561198312638244,233270,2
57630,76561198312638244,130,2
57631,76561198312638244,70,2
57632,76561198312638244,362890,2


In [46]:
# Se crea el reader del modelo para parsear los datos necesarios para la
# recomendación. En este caso, la escala de rating de recomendación se basa en
# "sentiment_analysis", donde el menor dato es 0 y el mayor es 2.
reader = Reader(rating_scale=[0,2])

# Se instancian los datos que se usarán para el modelo de recomendación.
datos = Dataset.load_from_df(r,reader)

In [47]:
# Se crea el modelo de recomendación.
modelo = SVD()

In [48]:
# Se observa una prueba de los resultados del modelo.
cross_validate(modelo,datos,measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6383  0.6460  0.6397  0.6284  0.6378  0.6380  0.0056  
MAE (testset)     0.5558  0.5587  0.5551  0.5490  0.5553  0.5548  0.0032  
Fit time          0.84    0.84    1.03    0.82    1.25    0.96    0.16    
Test time         0.10    0.08    0.53    0.05    0.09    0.17    0.18    


{'test_rmse': array([0.63829211, 0.64600575, 0.63966516, 0.62844745, 0.63780912]),
 'test_mae': array([0.55582964, 0.55870059, 0.55509516, 0.54900638, 0.55533642]),
 'fit_time': (0.8432180881500244,
  0.8409066200256348,
  1.0314154624938965,
  0.8243811130523682,
  1.245262861251831),
 'test_time': (0.09902572631835938,
  0.08204793930053711,
  0.5340762138366699,
  0.052289724349975586,
  0.09196114540100098)}

In [34]:
# Se entrena el modelo de recomendación.
entrenamiento = datos.build_full_trainset()
modelo.fit(entrenamiento)

In [55]:
# Se guarda el modelo en un archivo para poder usarlo posteriormente
dump.dump("/content/modelo_entrenado.pkl",algo=modelo)

In [60]:
# Se revisa que el modelo cargue adecuadamente.
modelo = dump.load("/content/modelo_entrenado.pkl")[1]

In [61]:
# Se crea una primera prueba para obtener las recomendaciones.
def recomendacion(user_id:str, n=5):
    model = modelo
    # Obtén todos los ítems que el usuario no ha calificado
    items_unrated = [item for item in entrenamiento.all_items() if item not in entrenamiento.ur[user_id]]

    # Haz predicciones para los ítems no calificados por el usuario
    predictions = [model.predict(user_id, item) for item in items_unrated]

    # Ordena las predicciones en orden descendente de calificación
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Devuelve las mejores 'n' recomendaciones
    top_n = predictions[:n]

    return top_n

In [62]:
# Se observa el resultado.
recomendacion("js41637")

[Prediction(uid='js41637', iid=2600, r_ui=None, est=1.8851557568377737, details={'was_impossible': False}),
 Prediction(uid='js41637', iid=2420, r_ui=None, est=1.7867207368152782, details={'was_impossible': False}),
 Prediction(uid='js41637', iid=440, r_ui=None, est=1.7840001841270614, details={'was_impossible': False}),
 Prediction(uid='js41637', iid=380, r_ui=None, est=1.781266289907966, details={'was_impossible': False}),
 Prediction(uid='js41637', iid=280, r_ui=None, est=1.724522609483712, details={'was_impossible': False})]